In [ ]:
def query(x) :
    return -1 * (x - 7)**2 + 49

def find_peak(n) :
    low, high = 0, n
    while (low < high) :
        mid = (low + high) // 2
        if (query(mid) < query(mid+1)) :
            low = mid + 1
        else :
            high = mid
    return high

print(f"{find_peak(200)} is the peak")

# Peak: 7

In [ ]:
import random

tasks = {
    'time': [5, 8, 4, 7, 6, 3, 9],
    'cost': [
        [10, 12, 9],    # Task costs
        [15, 14, 16],   
        [8, 9, 7],      
        [12, 10, 13],   
        [14, 13, 12],   
        [9, 8, 10],     
        [11, 12, 13]    
    ]
}

facilities = {
    'capacity': [24, 30, 28] 
}

# Genetic Algorithm Parameters
POPULATION_SIZE = 50
GENERATIONS = 100
CX_PROBABILITY = 0.8
MUT_PROBABILITY = 0.2

NUM_TASKS = len(tasks['time'])

def evaluate(individual):
    total_cost = 0
    facility_time = [0, 0, 0]  
    
    for task_idx, facility in enumerate(individual):
        facility_idx = facility - 1
        total_cost += tasks['cost'][task_idx][facility_idx]
        facility_time[facility_idx] += tasks['time'][task_idx]
    
    penalty = 0
    for i in range(3):
        if facility_time[i] > facilities['capacity'][i]:
            penalty += 1000 * (facility_time[i] - facilities['capacity'][i])
    
    return total_cost + penalty

def generate_individual():
    return [random.randint(1, 3) for _ in range(NUM_TASKS)]

def crossover(parent1, parent2):
    if random.random() < CX_PROBABILITY:
        point = random.randint(1, NUM_TASKS - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    return parent1[:], parent2[:]

def mutate(individual):
    for i in range(NUM_TASKS):
        if random.random() < 0.1: 
            individual[i] = random.randint(1, 3)

def tournament_selection(population, fitnesses, k=3):
    selected = []
    for _ in range(len(population)):
        candidates = random.sample(list(zip(population, fitnesses)), k)
        selected.append(min(candidates, key=lambda x: x[1])[0])
    return selected

def main():
    random.seed(42)

    population = [generate_individual() for _ in range(POPULATION_SIZE)]

    fitnesses = [evaluate(ind) for ind in population]

    for gen in range(GENERATIONS):
        selected = tournament_selection(population, fitnesses)

        next_generation = []
        for i in range(0, POPULATION_SIZE, 2):
            parent1 = selected[i]
            parent2 = selected[i+1] if i+1 < POPULATION_SIZE else selected[0]
            child1, child2 = crossover(parent1, parent2)
            mutate(child1)
            mutate(child2)
            next_generation.extend([child1, child2])

        population = next_generation[:POPULATION_SIZE]
        fitnesses = [evaluate(ind) for ind in population]

    best_idx = fitnesses.index(min(fitnesses))
    best_solution = population[best_idx]
    best_cost = fitnesses[best_idx]

    return best_solution, best_cost

if __name__ == "__main__":
    best_solution, best_cost = main()

    print("\nOptimal Task Allocation:")
    for i, fac in enumerate(best_solution):
        cost = tasks['cost'][i][fac - 1]
        time = tasks['time'][i]
        print(f"Task {i+1} → Facility {fac} (Cost: {cost}, Time: {time} hrs)")

    # Calculate facility loads
    facility_loads = [0, 0, 0]
    for i, fac in enumerate(best_solution):
        facility_loads[fac - 1] += tasks['time'][i]

    print("\nFacility Loads:")
    for i, load in enumerate(facility_loads):
        print(f"Facility {i+1}: {load} hrs (Capacity: {facilities['capacity'][i]} hrs)")

    print(f"\nTotal Cost: {best_cost}")


## Optimal Task Allocation:
Task 1 → Facility 3 (Cost: 9, Time: 5 hrs)  
Task 2 → Facility 2 (Cost: 14, Time: 8 hrs)  
Task 3 → Facility 3 (Cost: 7, Time: 4 hrs)  
Task 4 → Facility 2 (Cost: 10, Time: 7 hrs)  
Task 5 → Facility 3 (Cost: 12, Time: 6 hrs)  
Task 6 → Facility 2 (Cost: 8, Time: 3 hrs)  
Task 7 → Facility 1 (Cost: 11, Time: 9 hrs)  

## Facility Loads:
Facility 1: 9 hrs (Capacity: 24 hrs)  
Facility 2: 18 hrs (Capacity: 30 hrs)  
Facility 3: 15 hrs (Capacity: 28 hrs)  

# Total Cost: 71

# Q3 CSP Codes:

In [12]:
import time # Imported to compare efficiency

time_taken = time.time()

def cross(A, B):
    return [a + b for a in A for b in B]

rows = 'ABCDEFGHI'
cols = '123456789'
cells = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
col_units = [cross(rows, c) for c in cols]
box_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + col_units + box_units
units = {s: [u for u in unitlist if s in u] for s in cells}
peers = {s: set(sum(units[s], [])) - {s} for s in cells}

def parse_grid(grid):
    values = {s: '123456789' for s in cells}
    for s, d in zip(cells, grid):
        if d in '123456789' and not assign(values, s, d):
            return False
    return values

def assign(values, s, d):
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    return False

def eliminate(values, s, d):
    if d not in values[s]:
        return values
    values[s] = values[s].replace(d, '')
    if len(values[s]) == 0:
        return False
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    for u in units[s]:
        dplaces = [s2 for s2 in u if d in values[s2]]
        if len(dplaces) == 0:
            return False
        elif len(dplaces) == 1:
            if not assign(values, dplaces[0], d):
                return False
    return values

def AC3(values):
    queue = [(xi, xj) for xi in cells for xj in peers[xi]]
    while queue:
        xi, xj = queue.pop()
        if revise(values, xi, xj):
            if len(values[xi]) == 0:
                return False
            for xk in peers[xi] - {xj}:
                queue.append((xk, xi))
    return True

def revise(values, xi, xj):
    revised = False
    for x in values[xi]:
        if all(x == y for y in values[xj]):
            values[xi] = values[xi].replace(x, '')
            revised = True
    return revised

def is_solved(values):
    return all(len(values[s]) == 1 for s in cells)

def select_unassigned_variable(values):
    unassigned = [(len(values[s]), s) for s in cells if len(values[s]) > 1]
    return min(unassigned)[1] if unassigned else None

def backtrack(values):
    if values is False:
        return False
    if is_solved(values):
        return values
    s = select_unassigned_variable(values)
    for d in values[s]:
        new_values = values.copy()
        result = assign(new_values, s, d)
        if result:
            ac3_result = AC3(result.copy())
            if ac3_result:
                attempt = backtrack(result)
                if attempt:
                    return attempt
    return False

def solve(grid):
    values = parse_grid(grid)
    if not values:
        return False
    AC3(values)
    result = backtrack(values)
    if result:
        return ''.join(result[s] for s in cells)
    return False

def main():
    with open("sudoku_input.txt", "r") as f:
        puzzles = f.read().strip().splitlines()
    with open("sudoku_output.txt", "w") as f:
        for puzzle in puzzles:
            solution = solve(puzzle)
            print(f"initial puzzle: {puzzle}\nSolved puzzle: {solution}\nTime Taken: {time.time() - time_taken}\n")
            f.write(solution + "\n")

if __name__ == "__main__":
    main()

initial puzzle: 003020600900305001001806400008102900700000008006708200002609500800203009005010300
Solved puzzle: 483921657967345821251876493548132976729564138136798245372689514814253769695417382
Time Taken: 0.01699376106262207



In [13]:
# GPT IMPLEMENTATION
# Creating a Sudoku solver implementation based on the assignment
# This version uses constraint propagation (AC3) and backtracking

import time
from typing import List, Dict

time_taken = time.time()
def cross(A: str, B: str) -> List[str]:
    return [a + b for a in A for b in B]

rows = 'ABCDEFGHI'
cols = '123456789'
cells = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
col_units = [cross(rows, c) for c in cols]
box_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + col_units + box_units
units = {s: [u for u in unitlist if s in u] for s in cells}
peers = {s: set(sum(units[s], [])) - {s} for s in cells}

def parse_grid(grid: str) -> Dict[str, str] | bool:
    values = {s: '123456789' for s in cells}
    for s, d in zip(cells, grid):
        if d in '123456789' and not assign(values, s, d):
            return False
    return values

def assign(values: Dict[str, str], s: str, d: str) -> Dict[str, str] | bool:
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    return False

def eliminate(values: Dict[str, str], s: str, d: str) -> Dict[str, str] | bool:
    if d not in values[s]:
        return values
    values[s] = values[s].replace(d, '')
    if len(values[s]) == 0:
        return False
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    for u in units[s]:
        dplaces = [s2 for s2 in u if d in values[s2]]
        if len(dplaces) == 0:
            return False
        elif len(dplaces) == 1:
            if not assign(values, dplaces[0], d):
                return False
    return values

def AC3(values: Dict[str, str]) -> bool:
    queue = [(xi, xj) for xi in cells for xj in peers[xi]]
    while queue:
        xi, xj = queue.pop()
        if revise(values, xi, xj):
            if len(values[xi]) == 0:
                return False
            for xk in peers[xi] - {xj}:
                queue.append((xk, xi))
    return True

def revise(values: Dict[str, str], xi: str, xj: str) -> bool:
    revised = False
    for x in values[xi]:
        if all(x == y for y in values[xj]):
            values[xi] = values[xi].replace(x, '')
            revised = True
    return revised

def is_solved(values: Dict[str, str]) -> bool:
    return all(len(values[s]) == 1 for s in cells)

def select_unassigned_variable(values: Dict[str, str]) -> str:
    unassigned = [(len(values[s]), s) for s in cells if len(values[s]) > 1]
    return min(unassigned)[1] if unassigned else None

def backtrack(values: Dict[str, str]) -> Dict[str, str] | bool:
    if values is False:
        return False
    if is_solved(values):
        return values
    s = select_unassigned_variable(values)
    for d in values[s]:
        new_values = values.copy()
        result = assign(new_values, s, d)
        if result:
            ac3_result = AC3(result.copy())
            if ac3_result:
                attempt = backtrack(result)
                if attempt:
                    return attempt
    return False

def solve(grid: str) -> str:
    values = parse_grid(grid)
    if not values:
        return ""
    AC3(values)
    result = backtrack(values)
    return ''.join(result[s] for s in cells) if result else ""

print(f"Time Taken: {time.time() - time_taken}")

Time Taken: 0.004998922348022461


In [5]:
from ortools.sat.python import cp_model
import time

time_taken = time.time()

def solve_sudoku_or_tools(grid_string):
    model = cp_model.CpModel()
    cell = {}
    for i in range(9):
        for j in range(9):
            cell[(i, j)] = model.NewIntVar(1, 9, f'cell_{i}_{j}')
    
    for i in range(9):
        model.AddAllDifferent([cell[(i, j)] for j in range(9)])  # Rows
        model.AddAllDifferent([cell[(j, i)] for j in range(9)])  # Columns

    for block_row in range(3):
        for block_col in range(3):
            block = []
            for i in range(3):
                for j in range(3):
                    block.append(cell[(block_row * 3 + i, block_col * 3 + j)])
            model.AddAllDifferent(block)  # Blocks

    for i in range(9):
        for j in range(9):
            char = grid_string[i * 9 + j]
            if char in '123456789':
                model.Add(cell[(i, j)] == int(char))
    
    solver = cp_model.CpSolver()
    start = time.time()
    status = solver.Solve(model)
    end = time.time()

    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        solution = ''
        for i in range(9):
            for j in range(9):
                solution += str(solver.Value(cell[(i, j)]))
        return solution, end - start
    else:
        return None, end - start
print(f"Time Taken: {time.time() - time_taken}")

Time Taken: 0.00099945068359375


## Q3 CSP Time comparisons

### MY CODE: Time Taken: 0.01699376106262207
### GPT CODE: Time Taken: 0.004998922348022461
### GOOGLE ORTOOLS CODE: Time Taken: 0.00099945068359375